# TestQuery Notebook

This is a test Jupyter notebook to accompany the post:

> <p><b>[Jupyter Notebooks with Elixir and RDF:<br/>Using IElixir in Jupyter Lab to explore SPARQL.ex](https://medium.com/@tonyhammond/jupyter-notebooks-with-elixir-and-rdf-598689c2dad3)
</b></p>

We shall be using [IElixir](https://github.com/pprzetacznik/IElixir) which provides the Jupyter kernel for Elixir. We'll also be using the Elixir package [SPARQL.ex](https://hex.pm/packages/sparql) which provides for SPARQL support. 

Here's the overview:

1. Setting up the environment
2. Simple queries
3. Install our TestQuery module
4. Testing it out
5. Etc.

![jupyter](../../priv/images/jupyter.png)


## 1. Setting up the environment

&#x279C; ** Create `sparql_env`.**

IElixir uses the concept of virtual environments for managing packages. It uses [`Boyle`](https://github.com/pprzetacznik/IElixir#package-management-with-boyle) as its package manager.

Let's first create a `sparql_env` environment for our SPARQL dependencies using `Boyle.mk/1`. And note that we have previously created an `rdf_env` environment for exploring [RDF.ex](https://hex.pm/packages/rdf).

In [1]:
Boyle.mk("sparql_env")

All dependencies up to date


{:ok, ["rdf_env", "sparql_env"]}

&#x279C; **Activate the environment.**

Next step is to activate the environment which will take care of compiling.

In [3]:
Boyle.activate("sparql_env")

All dependencies up to date


:ok

In [4]:
Boyle.list

{:ok, ["rdf_env", "sparql_env"]}

And we can double check that this is the current environment.

In [6]:
Boyle.active_env_name

"sparql_env"

&#x279C; **Install `sparql_client` dependencies.**

Next we install out dependencies.

In [ ]:
Boyle.install({:sparql_client, "~> 0.2.1"})

&#x279C; **And check our packages.**

And lastly let's just sanity check that we do indeed have the `SPARQL` and `SPARQL.Client` modules installed.

In [4]:
exports SPARQL

execute_query/2                   execute_query/3                   query/1                           
query/2                           result_format/1                   result_format_by_extension/1      
result_format_by_media_type/1     result_formats/0                  


In [5]:
exports SPARQL.Client

__adapter__/0               __middleware__/0            default_accept_header/1     
delete/1                    delete/2                    delete/3                    
delete!/1                   delete!/2                   delete!/3                   
get/1                       get/2                       get/3                       
get!/1                      get!/2                      get!/3                      
head/1                      head/2                      head/3                      
head!/1                     head!/2                     head!/3                     
options/1                   options/2                   options/3                   
options!/1                  options!/2                  options!/3                  
patch/2                     patch/3                     patch/4                     
patch!/2                    patch!/3                    patch!/4                    
post/2                      post/3                      post/4   

Looks good.

## 2. Simple queries

&#x279C; **Let's try out a simple query.**


Let's choose a SPARQL endpoint. The ever-reliable [DBpedia](https://wiki.dbpedia.org/) will be good.

In [2]:
service = "http://dbpedia.org/sparql"

"http://dbpedia.org/sparql"

And we'll create a simple SPARQL query.

In [1]:
query = "select * where {?s ?p ?o} limit 5"

"select * where {?s ?p ?o} limit 5"

&#x279C; **First query try gives error.**

In [10]:
SPARQL.Client.query(query, service)

{:error, "unsupported result format for select query: \"text/html; charset=UTF-8\""}

&#x279C; **Let's check out the documentation.**

In [ ]:
h SPARQL.Client.query

&#x279C; **Second try succeeds – after setting :request_method (and :protocol_version).**

In [ ]:
SPARQL.Client.query(query, service, request_method: :get, protocol_version: "1.1")

Hey, that works! Let's capture the result.

In [6]:
{:ok, result} = SPARQL.Client.query(query, service, request_method: :get, protocol_version: "1.1")

{:ok, %SPARQL.Query.Result{results: [%{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid-nullable>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, "s" => ~I<http://www.openlinksw.com/virtrdf-data-formats#default-iid-nonblank-nullable>}, %{"o" => ~I<http://www.openlinksw.com/schemas/virtrdf#QuadMapFormat>, "p" => ~I<http://www.w3.org/1999/02/22-rdf-syntax

In [11]:
result |> SPARQL.Query.Result.get(:p) |> IO.inspect

[~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,
 ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,
 ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,
 ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>,
 ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>]


[~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>, ~I<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>]

Time for something more interesting. Let's try out our earlier `TestQuery` package.

## 3. Installing our TestQuery module

&#x279C; **Install TestQuery**

Is our `TestQuery` module loaded?

In [ ]:
exports TestQuery

That'll be a no, then. So, let's explicitly import the module.

In [22]:
import_file("../../tonyhammond/examples/test_query/lib/test_query.ex")

Protocol.UndefinedError: 1

Rats!

Turns out that our call `:code.priv_dir(:test_query)}` is failing.

Let's just hardwire that for now and try again.

In [22]:
import_file("../../tonyhammond/examples/test_ipynb/lib/test_query.ex")

{:module, TestQuery, <<70, 79, 82, 49, 0, 0, 7, 104, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 0, 191, 0, 0, 0, 18, 16, 69, 108, 105, 120, 105, 114, 46, 84, 101, 115, 116, 81, 117, 101, 114, 121, 8, 95, 95, 105, 110, 102, ...>>, {:query, 2}}

And let's sanity check.

In [ ]:
exports TestQuery

Success!

&#x279C; **Install TestQuery.Client.** 

Now let's import the `TestQuery.Client` module.

In [23]:
import_file("../../tonyhammond/examples/test_query/lib/test_query/client.ex")

Protocol.UndefinedError: 1

Same problem. Same fix.

In [23]:
import_file("../../tonyhammond/examples/test_ipynb/lib/test_query/client.ex")

{:module, TestQuery.Client, <<70, 79, 82, 49, 0, 0, 26, 80, 66, 69, 65, 77, 65, 116, 85, 56, 0, 0, 2, 223, 0, 0, 0, 71, 23, 69, 108, 105, 120, 105, 114, 46, 84, 101, 115, 116, 81, 117, 101, 114, 121, 46, 67, 108, 105, 101, 110, ...>>, {:_read_tuple, 1}}

And again sanity check.

In [ ]:
exports TestQuery.Client

## 4. Testing it out

&#x279C; **Now let's query**

In [ ]:
o = SPARQL.Query.Result.get(result, :o) |> List.first

In [ ]:
query

In [ ]:
exports SPARQL.Query.Result

In [ ]:
SPARQL.Query.Result.get(result, :o) 

In [ ]:
o.value

In [ ]:
rq = File.read!("/Users/tony/Projects/local/elixir/examples/test_super/priv/queries/dbpedia_query.rq")

In [ ]:
IO.puts rq

## 5. Etc.